## Make the non_pprm dataset

This makes a sample of words that have not been identified as pragmatic markers

In [3]:
import json
import re
import random

# Load the transcripts and phrases from JSON files
with open('data/news_dialogue_large_sample.json', 'r') as file:
    transcripts = json.load(file)

with open('data/pprms/authority_markers.json', 'r') as file:
    search_terms = json.load(file)

with open('data/pprms/hedging_markers.json', 'r') as file:
    search_terms +=json.load(file)

def filter_non_matching_utterances(transcripts, terms):
    # Function to filter utterances that do not match any of the terms
    results = []
    for transcript in transcripts:
        previous_utterance = None
        for current_index, utterance in enumerate(transcript['utt']):
            matched = False  # Track if any term matches
            # Check each term separately
            for term in terms:
                if re.search(rf"\b{re.escape(term)}\b", utterance, re.IGNORECASE):
                    matched = True
                    break  # Break the loop if any term matches
            # If no terms matched, collect the utterance
            if not matched:
                result = {
                    "transcript_id": transcript['id'],
                    "previous_statement": previous_utterance if previous_utterance else "None",
                    "statement": utterance
                }
                results.append(result)
            # Update the previous utterance
            previous_utterance = utterance
    return results

# Get the filtered list of non-matching utterances
non_matching_utterances = filter_non_matching_utterances(transcripts, search_terms)

# Sample 100 entries randomly from the results if there are more than 100
sampled_non_matching_utterances = random.sample(non_matching_utterances, min(100, len(non_matching_utterances)))

# Save the non-matching utterances to a JSON file
output_file_path = 'data/non_matching_utterances.json'
with open(output_file_path, 'w') as file:
    json.dump(non_matching_utterances, file, indent=4)

output_file_path = 'data/non_matching_utterances_sample.json'
with open(output_file_path, 'w') as file:
    json.dump(sampled_non_matching_utterances, file, indent=4)

# Optionally, you can print the JSON to check the output
# print(json.dumps(sampled_non_matching_utterances, indent=4))


In [5]:
# Sample 100 entries randomly from the results if there are more than 100
sampled_non_matching_utterances = random.sample(non_matching_utterances, min(100, len(non_matching_utterances)))

# Save the non-matching utterances to a JSON file
output_file_path = 'data/non_matching_utterances.json'
with open(output_file_path, 'w') as file:
    json.dump(non_matching_utterances, file, indent=4)

output_file_path = 'data/non_matching_utterances_sample.json'
with open(output_file_path, 'w') as file:
    json.dump(sampled_non_matching_utterances, file, indent=4)

# print("Filtered Utterances:", len(sampled_non_matching_utterances))

In [6]:
print("Filtered Utterances:", len(non_matching_utterances))
print("Filtered Utterances:", len(sampled_non_matching_utterances))

Filtered Utterances: 49761
Filtered Utterances: 100


In [4]:
import json

# Load your initial JSON data and the graded dataset
with open('data/filtered_utterances.json', 'r') as file:
    json_data = json.load(file)

# Define the list of markers
with open('data/short_prm_list.json', 'r') as merge_file:
    markers = json.load(merge_file)

with open('data/filtered_utterances_samples_MW_HNEB_MERGE.json', 'r') as merge_file:
    graded_data = json.load(merge_file)

with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_1.json', 'r') as merge_file2:
    graded_data2 = json.load(merge_file2)

with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_2.json', 'r') as merge_file3:
    graded_data3 = json.load(merge_file3)

with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_3.json', 'r') as merge_file3:
    graded_data4 = json.load(merge_file3)

full_grade_data = graded_data+graded_data2+graded_data3+graded_data4

# Extract statements from graded data for exclusion
# graded_statements = {item['statement'] for item in full_grade_data}
graded_statements = {item['transcript_id'] + " - " + item['previous_statement'] for item in full_grade_data}

print(len(json_data))
print(len(graded_statements))
print(len(markers))
# i=0
# for k in graded_statements:
#     if i == 2:
#         break
#     else:
#         print(k)
#         i+=1

70423
400
88


In [5]:
# Initialize the results list
ungraded_data = []
i = 0
j=0
# Filter the data
for sample in json_data:
    # Check if the statement has been graded already
    j+=1
    sample_statement = sample['transcript_id'] + " - " + sample['previous_statement']
    # print(sample_statement)
    if sample_statement not in graded_statements:
        if sample_statement == "CNN-412394 - It's important say over and over and over and over and over again on this day, we wish the president well and we wish him a speedy recovery but it is just a fact, it is just a fact that his behavior, the way he has conducted himself in recent days and weeks is directly contrary to the advice of his own experts.":
            print("sample_statement mistakenly added:", sample_statement)
        # Only add the sample if there are any filtered terms
        ungraded_data.append(sample)
        if j%10000==0:
            print(f"Filtered {j} samples: {sample}")
    else:
        if sample_statement == "CNN-412394 - It's important say over and over and over and over and over again on this day, we wish the president well and we wish him a speedy recovery but it is just a fact, it is just a fact that his behavior, the way he has conducted himself in recent days and weeks is directly contrary to the advice of his own experts.":
            print("sample_statement correctly skipped added:", sample_statement)
        i +=1
        if i > 290:
            # if i%10==0:
            print(f"Graded Statement Skipped {i}: ", sample['statement'])

print("Total Skipped Examples:", i)
print("Total Processed Examples:", j)
# Output the filtered results as JSON
with open('data/new_filtered_utterances.json', 'w') as output_file:
    json.dump(ungraded_data, output_file, indent=2)


Filtered 10000 samples: {'transcript_id': 'CNN-21637', 'matched_terms': {'of course': 'ungraded', 'sure': 'ungraded', 'clear': 'ungraded', 'obviously': 'ungraded'}, 'previous_statement': 'And, Eileen, as they do go about considering that, are they considering more than legal options? Are they checking in with Democratic supporters and leaders within the party to see how much support really there would be out there for continued fight?', 'statement': "Right now, the only information I have is that they've been talking amongst themselves with aides. I'm <SURE> they're getting some input <OF COURSE> from Democrats. They've been told from the Democrats that -- from major Democratic Party officials that they will support anything the vice president does. <OBVIOUSLY>, he's under increasing public and political pressure to end this thing, and he's already said that if the decision was <CLEAR> to him, after all of this, these legal wrangles and the court decision was clear to him, that the ele

### Created Groups of 100, starting with 4

In [9]:
import math
import random
import json

last_completed_file = 3

# Number of samples per file
random.shuffle(ungraded_data)

samples_per_file = 100
total_samples = len(ungraded_data)
total_files = math.ceil(total_samples / samples_per_file)

r = 20 - last_completed_file
print(r)
# Generate a list of indices in random order
indices = list(range(r))

# Split data into chunks and save to separate files using shuffled indices
for i in indices:
    start_index = i * samples_per_file
    end_index = start_index + samples_per_file
    chunk = ungraded_data[start_index:end_index]
    
    # Generate filename based on the chunk number
    filename = f'data/new_filtered_utterances_to_grade/filtered_data_HN_part_{i + last_completed_file + 1}.json'
    
    # Save the chunk to a file
    with open(filename, 'w') as file:
        json.dump(chunk, file, indent=2)
    filename = f'data/new_filtered_utterances_to_grade/filtered_data_MW_part_{i + last_completed_file + 1}.json'
    print("Saved File:", i+last_completed_file+1)
    # Save the chunk to a file
    with open(filename, 'w') as file:
        json.dump(chunk, file, indent=2)



print(f"Data split into {20} files with up to {samples_per_file} samples each.")

17
Saved File: 4
Saved File: 5
Saved File: 6
Saved File: 7
Saved File: 8
Saved File: 9
Saved File: 10
Saved File: 11
Saved File: 12
Saved File: 13
Saved File: 14
Saved File: 15
Saved File: 16
Saved File: 17
Saved File: 18
Saved File: 19
Saved File: 20
Data split into 20 files with up to 100 samples each.


In [18]:
import json

# Sample JSON data
# with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_2.json', 'r') as prev_grade:
#     prev_grade2 = json.load(prev_grade)
with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_3.json', 'r') as prev_grade:
    prev_grade3 = json.load(prev_grade)
with open('data/new_filtered_utterances_to_grade/filtered_data_MW_part_4.json', 'r') as prev_grade:
    prev_grade4 = prev_grade3+json.load(prev_grade)

# Dictionary to store the count of each term for each category
term_count = {}

# Iterate through each item in the data
for item in prev_grade4:
    matched_terms = item['matched_terms']
    for term, category in matched_terms.items():
        # Initialize the term dictionary if not already present
        if term not in term_count:
            term_count[term] = {}
        # Initialize the category count if not already present
        if category not in term_count[term]:
            term_count[term][category] = 0
        # Increment the count for this category under the term
        term_count[term][category] += 1


sorted_term_count = dict(sorted(term_count.items(), key=lambda item: item[1].get('none', 0), reverse=True))

# Output the term count dictionary
# print(json.dumps(term_count, indent=2))
for item in sorted_term_count:
    # print(item, term_count[item])
    values = term_count[item]
    sample_count = 0
    none_count = 0
    for k, v in values.items():
        sample_count+=v
        if k == "none":
            none_count += v
    print(f"{item}: none percentage: {none_count/sample_count*100:.2f}% \n * h/a_count: {sample_count-none_count}\n * none_count: {none_count}\n")


know: none percentage: 51.61% 
 * h/a_count: 30
 * none_count: 32

about: none percentage: 91.18% 
 * h/a_count: 3
 * none_count: 31

well: none percentage: 89.66% 
 * h/a_count: 3
 * none_count: 26

could: none percentage: 57.14% 
 * h/a_count: 12
 * none_count: 16

think: none percentage: 22.92% 
 * h/a_count: 37
 * none_count: 11

around: none percentage: 83.33% 
 * h/a_count: 2
 * none_count: 10

thought: none percentage: 80.00% 
 * h/a_count: 2
 * none_count: 8

believe: none percentage: 53.85% 
 * h/a_count: 6
 * none_count: 7

should: none percentage: 35.29% 
 * h/a_count: 11
 * none_count: 6

feel: none percentage: 83.33% 
 * h/a_count: 1
 * none_count: 5

best: none percentage: 100.00% 
 * h/a_count: 0
 * none_count: 5

sure: none percentage: 44.44% 
 * h/a_count: 5
 * none_count: 4

may: none percentage: 20.00% 
 * h/a_count: 16
 * none_count: 4

thinking: none percentage: 100.00% 
 * h/a_count: 0
 * none_count: 4

hope: none percentage: 100.00% 
 * h/a_count: 0
 * none_count

In [19]:
# Dictionary to store the count of each category across all terms
category_count = {}

# Iterate through each item in the data
for item in prev_grade4:
    matched_terms = item['matched_terms']
    for term, category in matched_terms.items():
        # Initialize the category count if not already present
        if category not in category_count:
            category_count[category] = 0
        # Increment the count for this category
        category_count[category] += 1

# Sort the categories based on their count, descending
sorted_category_count = dict(sorted(category_count.items(), key=lambda item: item[1], reverse=True))

# Output the sorted category count
for category, count in sorted_category_count.items():
    if category == "hedge":
        hedge = count
    if category == "authority":
        auth = count
    if category == "none":
        none = count
    print(f"Category: {category}, Count: {count}")
print(f"Total: {hedge+auth+none}, None Rate: {none/(hedge+auth+none)*100:.2f}%")

Category: none, Count: 198
Category: hedge, Count: 163
Category: authority, Count: 73
Total: 434, None Rate: 45.62%
